# Time-series signatures from raw brdf imagery / smoothed ts outputs

In [ ]:
import os
import sys
from pathlib import Path
import rasterio as rio
from rasterio import plot
import matplotlib.pyplot as plt
import shutil
import tempfile
import json
import random
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import Proj, transform
from pyproj import CRS
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon
from rasterio.plot import show
import xarray as xr

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from ts_profile import *

In [ ]:
##### Note: 
###  Notebook is housed on login node; time series calculations shold not be run on login node (and take a long time anyway). 
###  Better to install on cluster, run GettimeSeries.sh in Bash folder and copy output file path to parameters 
###  to read with `LoadTSfromFile(TSfile)` to load time series here for plotting.

###TODO: Enable editing Bash script to run through Slurm via this notebook directly

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basic_config
print("Run Parameters: \n raw_dir = {} \n grid_cell = {} \n index_dir = {} \n grid_file = {} \n local_dir = {}"
      .format(basic_config['raw_dir'],basic_config['grid_cell'],basic_config['index_dir'],basic_config['grid_file'],basic_config['local_dir']))
%store -r timeseries_params
print(" point_file = {}".format(basic_config['ptfile']))
print(" poly_file = {}".format(basic_config['polyfile']))
if timeseries_params['load_prerunTS'] == True:
    print(" Smooth TS loaded from {}".format(timeseries_params['smooth_TSfile']))
    print(" Raw TS loaded from {}".format(timeseries_params['raw_TSfile']))
    

In [ ]:
natpts = get_pts_in_grid(basic_config['grid_file'], basic_config['grid_cell'], basic_config['ptfile'])
natpts.head(n=5)

## Plot smooth time series

In [ ]:
if timeseries_params['load_prerunTS'] == True:
    if timeseries_params['smooth_TSfile']:
        smooth = load_ts_from_file(timeseries_params['smooth_TSfile'])
else:
    if basic_config['ptfile'].endswith('SelectedCoords.csv'):
        gcells = [basic_config['grid_cell']]
        load_samp = True
    else:
        gcells = basic_config['grid_cells']
        load_samp=timeseries_params['load_samp']
        
    smooth = get_timeseries_for_pts_multicell(basic_config['local_dir'], basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['smooth_dir'], 'Smooth', basic_config['grid_file'], gcells, 
             ground_polys=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp = load_samp, ptfile=basic_config['ptfile'])

In [ ]:
smooth = smooth[smooth['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
#plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Smoothed {} for sample {}-{}'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']))
plt.ylim(0,7000)
plt.show();

## To compare two smoothed time-series (i.e. with different parameters)
One is in the regular time_series folder called 'brdf_ts/ms/index' and the other is in folder called 'brdf_ts_old/ms/index'  

In [ ]:
if not os.path.exists(os.path.join(basic_config['smooth_dir'],'{:06d}'.format(basic_config['grid_cell']),'brdf_ts_old','ms',basic_config['spec_index'])):
    print('There is no old time series to compare')
else:
    if basic_config['ptfile'].endswith('SelectedCoords.csv'):
        gcells = [basic_config['grid_cell']]
        load_samp = True
    else:
        gcells = basic_config['grid_cells']
        load_samp=timeseries_params['load_samp']
        
    old = get_timeseries_for_pts_multicell(basic_config['local_dir'], basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['smooth_dir'], 'Smooth_old', basic_config['grid_file'], gcells, 
            ground_polys=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp, basic_config['ptfile'])

    old = old[old['ALL'] != 0]

    fig = plt.figure(figsize=(10, 5))
    plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
    plt.errorbar(old.index, old['ALL'], yerr=old['stdv'], fmt='o', color='g')
    plt.title('New vs old smoothed ts {} for sample {}-{}'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']))
    plt.ylim(0,7000)
    plt.show();

## Plot raw time series

In [ ]:
if timeseries_params['load_prerunTS'] == True:
    if timeseries_params['raw_TSfile']:
        raw = load_TSfromFile(timeseries_params['raw_TSfile'])
else:
    raw = get_timeseries_for_pts_multicell(basic_config['home_dir'], basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['raw_dir'], timeseries_params['image_type'], basic_config['grid_file'], timeseries_params['grid_cells'], 
            ground_polys=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp=timeseries_params['load_samp'], ptfile=basic_config['ptfile'])

In [ ]:
raw = raw[raw['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
#plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Raw {} for sample points {}-{}'.format(basic_config['spec_index'], timeseries_params['start_yr'],timeseries_params['end_yr']))
plt.show()

## To plot a timeseries for each polygon in file:

In [ ]:
smooth = get_timeseries_for_pts_multicell(basic_config['home_dir'], basic_config['spec_index'], timeseries_params['start_yr'], timeseries_params['end_yr'],
            basic_config['smooth_dir'], 'Smooth', basic_config['grid_file'], timeseries_params['grid_cells'], 
            ground_polys=basic_config['polyfile'], oldest=timeseries_params['oldest_samp'], newest=timeseries_params['newest_samp'],
            npts=timeseries_params['npts'], seed=timeseries_params['seed1'], load_samp=timeseries_params['load_samp'], basic_config['ptfile'])

In [ ]:
smooth

In [ ]:
smooth = smooth[smooth['ALL'] != 0]

fig = plt.figure(figsize=(10, 5))
#plt.errorbar(smooth.index, smooth['ALL'], yerr=smooth['stdv'], fmt='o', color='k')
plt.plot(smooth.index, smooth[22], color='k')
#plt.errorbar(raw.index, raw['ALL'], yerr=raw['stdv'], fmt='o', color='g')
plt.title('Smoothed {} for natural forest points {}-{}'.format(basic_config['spec_index'],timeseries_params['start_yr'],timeseries_params['end_yr']))
plt.show();

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

out_name = str(basic_config['country']+'2_TimeSeries'+str(basic_config['grid_cell'])+'_from_'+str(timeseries_params['start_yr'])+'_to_'+str(timeseries_params['end_yr']))
!jupyter nbconvert --output-dir='../outputs' --to html --no-input --output=$out_name 2_TimeSeriesSignatures.ipynb